In [ ]:
# Get trainng and testing data
def split_file(input_file, train_file, test_file, train_ratio=0.9):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    random.shuffle(lines)

    split_index = int(len(lines) * train_ratio)

    # do the split
    train_lines = lines[:split_index]
    test_lines = lines[split_index:]

    with open(train_file, 'w') as file:
        file.writelines(train_lines)

    with open(test_file, 'w') as file:
        file.writelines(test_lines)


In [ ]:
input_file = 'reformatted.txt'
train_file = 'train.txt'
test_file = 'test.txt'

# Split the file
split_file(input_file, train_file, test_file)

print(f"Train and test files have been created: {train_file}, {test_file}")
from google.colab import files
files.download('train.txt')
files.download('test.txt')
print("Train and test files have been downloaded")

Train and test files have been created: train.txt, test.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Train and test files have been downloaded


In [ ]:
!pip install accelerate
!pip install chardet
!pip install TFTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
import numpy as np
import accelerate
import chardet
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline


In [ ]:
pip install torch

In [ ]:
import torch
import random

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='train.txt',
    block_size=128
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
# Training args
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=3,
    save_steps=10_000,
    save_total_limit=2,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.666700
1000,3.440900
1500,3.344300
2000,3.292000
2500,3.242300
3000,3.213300
3500,3.178000
4000,3.153500
4500,3.130800
5000,3.112400


TrainOutput(global_step=61059, training_loss=2.7437391148523482, metrics={'train_runtime': 9271.9938, 'train_samples_per_second': 19.756, 'train_steps_per_second': 6.585, 'total_flos': 1.1965672636416e+16, 'train_loss': 2.7437391148523482, 'epoch': 3.0})

In [ ]:
model_save_path = './gpt2_lyrics_model'

model.save_pretrained(model_save_path)

tokenizer_save_path = './gpt2_lyrics_tokenizer'
tokenizer.save_pretrained(tokenizer_save_path)

('./gpt2_lyrics_tokenizer/tokenizer_config.json',
 './gpt2_lyrics_tokenizer/special_tokens_map.json',
 './gpt2_lyrics_tokenizer/vocab.json',
 './gpt2_lyrics_tokenizer/merges.txt',
 './gpt2_lyrics_tokenizer/added_tokens.json')

In [ ]:
model_save_path = './gpt2_lyrics_model'
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer_save_path = './gpt2_lyrics_tokenizer'
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_save_path)

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# try resizing
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

print("Model pad_token_id:", model.config.pad_token_id)

# gpu was bugging for some reason
device = "cpu"
model.to(device)

Model pad_token_id: 50256


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import json


In [ ]:
def update_max_length(config_file, new_max_length):
    with open(config_file, 'r') as file:
        config = json.load(file)

    config['max_length'] = new_max_length

    with open(config_file, 'w') as file:
        json.dump(config, file, indent=4)

config_file = 'gpt2_lyrics_model/generation_config.json'
new_max_length = 80

# more length
update_max_length(config_file, new_max_length)

print(f"Updated max_length to {new_max_length} in {config_file}")

Updated max_length to 80 in gpt2_lyrics_model/generation_config.json


In [ ]:
def extract_input_text(file_path):
    input_texts = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[:2500]:
            parts = line.strip().rsplit(', ', 2)
            if len(parts) == 3:
                clue, length, _ = parts
                input_text = f"{clue}, {length},"
                input_texts.append(input_text)
    return input_texts


In [ ]:
def extract_answers(data):
    answers = []
    for line in data.strip().split('\n'):
        parts = line.rsplit(', ', 1)  # Split only on the last comma and space
        if len(parts) == 2:
            answer = parts[1].strip()
            answers.append(answer)
    return answers[0]


In [ ]:
def get_answers(inputs):

  generated_answers = []
  for input_text in inputs:
    length = len(input_text)
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(
        inputs["input_ids"],
        max_new_tokens=length + 10,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    final = extract_answers(generated_text)
    generated_answers.append(final)

  return generated_answers


In [ ]:
input_texts = extract_input_text('test.txt')
print(input_texts)
get_answers(input_texts)

['Author who wrote "Some day you will be old enough to start reading fairy tales again", 7,', 'Alphabetic trio, 3,', '14 years before the Magna Carta, 4,', 'West Coast air hub, for short, 3,', 'Awkward period, often, 5,', 'Comic Martha, 4,', 'Inc., overseas, 3,', 'Woolen caps, 4,', 'Tulsa sch., 3,', 'Least merry, 7,', 'Sch. near Beverly Hills, 4,', 'Bee-fitting?, 5,', 'Take too long, 7,', "Perry's creator, 4,", 'Shiny fabric, 4,', 'Credit ___ (banking giant), 6,', 'Prize higher than plata or bronce, 3,', 'Fashion no-no with a flip-flop, 4,', 'Moon unit?, 4,', 'In need of a chill pill, 7,', 'Jul. 4 happening, 3,', 'Get stuff ready to go, 6,', 'Offer you might have less interest in, for short?, 4,', 'It officially recognized Isr. in 1988, 3,', 'Bowl over, 6,', 'Waterfall, 5,', 'Tahoe, for one, 3,', 'Lawyer, at court, 7,', "Bald Mountain's range, 5,", '"... ___ quit!", 3,', 'Really, really big, 7,', 'Holiday driver, in a phrase, 6,', 'Abbr. sometimes seen twice in a row, 3,', 'Lunched, 3,

['LESTERT',
 'RST',
 'CCCP',
 'LGA',
 'SPREE',
 'RAYE',
 'LTD',
 'TAMS',
 'ORU',
 'SADEST',
 'UCLA',
 'TONIC',
 'OVERUSE',
 'CAPP',
 'LAME',
 'PONTI',
 'OBI',
 'ELLE',
 'LENS',
 'SWEETEN',
 'MLK',
 'REPAIR',
 'REPO',
 'UAE',
 'BLEARY',
 'SPREE',
 'ALP',
 'ATTORNEY',
 'OSAGE',
 'DOI',
 'BIGBANG',
 'HOMERS',
 'ETC',
 'ATE',
 'ONEPART',
 'CTR',
 'TAT',
 'BLOC',
 'SCARE',
 'ETES',
 'EON',
 'SOUSED',
 'ATM',
 'LTRS',
 'EATAT',
 'TOSSED',
 'SENSE',
 'NANA',
 'LILLE',
 'SPAR',
 'LIME',
 'SATANS',
 'TOTEM',
 'OVALS',
 'AVIA',
 'EASE',
 'SLEEPER',
 'ERAS',
 'ELDER',
 'ADELE',
 'STARE',
 'REAP',
 'REPS',
 'WEAL',
 'OLDAS',
 'ELATE',
 'ATOLL',
 'ESSEN',
 'BIOS',
 'PEG',
 'TSK',
 'APR',
 'STEREOS',
 'LORNE',
 'EMS',
 'POSE',
 'EDSEL',
 'SEENAT',
 'ERGO',
 'ICE',
 'JAMESBACHEL',
 'OAT',
 'ISEE',
 'REHEAT',
 'GETIT',
 'OMAN',
 'ARARAT',
 'IONIA',
 'NERD',
 'AERIAL',
 'CCNY',
 'ONCE',
 'TAPE',
 'ETERNAL',
 'TIEDBAR',
 'TACOIT',
 'EATUP',
 'OGRE',
 'AMC',
 'TENORES',
 'ACID',
 'TONY',
 'MER',
 'SORE',

In [ ]:
# Get correct answers

def get_actual_answers(filepath):
  answers = []
  with open(filepath, 'r') as file:
          lines = file.readlines()
          for line in lines[:2500]:
              parts = line.strip().rsplit(', ', 2)
              if len(parts) == 3:
                  clue, length, ans = parts
                  answers.append(ans)

  return answers


In [ ]:
actual_answers = get_actual_answers('test.txt')

In [ ]:
def accuracy(actual_answers, generated_answers):
    array1 = np.array(actual_answers)
    array2 = np.array(generated_answers)
    matches = np.sum(array1 == array2)

    return matches / len(array1)

In [ ]:
filepath = 'test.txt'
inputs = extract_input_text(filepath)
generated_answers = get_answers(inputs)
actual_answers = get_actual_answers(filepath)
accuracy(actual_answers, generated_answers)

0.1748

In [ ]:
print(actual_answers)

['CSLEWIS', 'OPQ', 'MCCI', 'SFO', 'TEENS', 'RAYE', 'CIE', 'TAMS', 'ORU', 'GRAVEST', 'UCLA', 'APIAN', 'ENDLATE', 'ERLE', 'LAME', 'SUISSE', 'ORO', 'SOCK', 'REAR', 'UPTIGHT', 'BBQ', 'PACKUP', 'REFI', 'PLO', 'DAZZLE', 'SAULT', 'SUV', 'PLEADER', 'UINTA', 'ORI', 'IMMENSE', 'BUSMAN', 'ETC', 'ATE', 'PGRATED', 'CTR', 'END', 'BLOC', 'MOOLA', 'ETES', 'EON', 'SEALED', 'IRA', 'TSPS', 'TASTE', 'IRONIC', 'DECODE', 'DOUG', 'HAVRE', 'EPEE', 'PUCE', 'THRICE', 'LAPEL', 'ASSES', 'NAPA', 'EASE', 'WANNABE', 'MORE', 'ALTAR', 'MAMAS', 'TIARA', 'KEEP', 'LABS', 'TACT', 'TSARS', 'SEND', 'SALEM', 'RONCO', 'SPAM', 'SKI', 'TSK', 'APR', 'STILLS', 'CYRUS', 'CCS', 'BOSS', 'ASCOT', 'PEERED', 'ELSE', 'CON', 'ERROLFLYNN', 'CUT', 'ORME', 'RESEAL', 'GETME', 'OMAN', 'ANKARA', 'GENOA', 'SIMP', 'CHORAL', 'GRIN', 'ONTO', 'TRAY', 'NOSHOW', 'NIGHTIE', 'ALTOID', 'DETER', 'NINE', 'AMC', 'PENCILS', 'PEAR', 'SPIN', 'LIA', 'SHOT', 'GALA', 'MATS', 'LAND', 'SIMOOM', 'ATTU', 'ARSENIC', 'AWNING', 'FEATS', 'ORR', 'ELEPHANTTRAINER', 'FRAME

In [ ]:
# calculate by letter accuracy
def letter_accuracy(words, guesses):
    correct_letters, total_letters = 0, 0
    for i in range(len(words)):
        word, guess = words[i], guesses[i]
        # null guesses
        if guess == "NULL":
            total_letters += len(word)
        else:
            # correct guess
            if word == guess:
                correct_letters += len(word)
                total_letters += len(word)

            else:
                # Case 1: guess too short, adding padding
                if len(word) > len(guess):
                    while len(guess) < len(word):
                        guess += '!'
                # Case 2: guess too long, crop to len(word)
                elif len(word) < len(guess):
                    guess = guess[:len(word)]

                # Word, Guess now guaranteed to be same length
                for i in range(len(word)):
                    if word[i] == guess[i]:
                        total_letters += 1
                        correct_letters += 1
                    else:
                        total_letters += 1
    return correct_letters, total_letters

In [ ]:
correct, total = letter_accuracy(actual_answers, generated_answers)
print(f"Accuracy is {correct/total}")

Accuracy is 0.24416472157385796


In [ ]:
set_word_len = set([len(word) for word in actual_answers])
for length in set_word_len:
    idxs = [i for i in range(len(actual_answers)) if len(actual_answers[i]) == length]
    subgroup_words = [actual_answers[i] for i in idxs]
    subgroup_guesses = [generated_answers[i] for i in idxs]
    correct, total = letter_accuracy(subgroup_words, subgroup_guesses)
    print(f"Correct {length}-letter prediction accuracy: {correct / total}")

Correct 3-letter prediction accuracy: 0.4401041666666667
Correct 4-letter prediction accuracy: 0.3368146214099217
Correct 5-letter prediction accuracy: 0.22907180385288967
Correct 6-letter prediction accuracy: 0.1393188854489164
Correct 7-letter prediction accuracy: 0.14126984126984127
Correct 8-letter prediction accuracy: 0.09970238095238096
Correct 9-letter prediction accuracy: 0.0873015873015873
Correct 10-letter prediction accuracy: 0.07058823529411765
Correct 11-letter prediction accuracy: 0.09090909090909091
Correct 12-letter prediction accuracy: 0.0
Correct 13-letter prediction accuracy: 0.07692307692307693
Correct 14-letter prediction accuracy: 0.0
Correct 15-letter prediction accuracy: 0.075


In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [ ]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin
wv_from_bin = load_embedding_model()

[==================================================] 100.0% 252.1/252.1MB downloaded
Loaded vocab size 400000


NameError: name 'words' is not defined

In [ ]:
def cos_similarity_incl_null(words, guesses):
    cos_sim = []
    for i in range(len(words)):
        try:
            cos_sim.append(wv_from_bin.distance(words[i].lower(), guesses[i].lower()))
        except:
            cos_sim.append(0)
    return sum(cos_sim) / len(cos_sim)

def cos_similarity_excl_null(words, guesses):
    cos_sim = []
    for i in range(len(words)):
        try:
            cos_sim.append(wv_from_bin.distance(words[i].lower(), guesses[i].lower()))
        except:
            pass
    if len(cos_sim):
        return sum(cos_sim) / len(cos_sim)
    else:
        return 0

In [ ]:
print(f"Cosine similarity w/ 0s for null guesses: {cos_similarity_incl_null(actual_answers, generated_answers)}")
print(f"Cosine similarity excluding null guesses: {cos_similarity_excl_null(actual_answers, generated_answers)}")

Cosine similarity w/ 0s for null guesses: 0.5232059581822716
Cosine similarity excluding null guesses: 0.6987259056921362


In [ ]:
for length in set_word_len:
    idxs = [i for i in range(len(actual_answers)) if len(actual_answers[i]) == length]
    subgroup_words = [actual_answers[i] for i in idxs]
    subgroup_guesses = [actual_answers[i] for i in idxs]
    print(f"Cosine similarity {length}-letter w/ 0s for null guesses: {cos_similarity_incl_null(subgroup_words, subgroup_guesses)}")
    print(f"Cosine similarity {length}-letter excluding null guesses: {cos_similarity_excl_null(subgroup_words, subgroup_guesses)}")
    print()

Cosine similarity 3-letter w/ 0s for null guesses: 5.3551048040390015e-09
Cosine similarity 3-letter excluding null guesses: 5.386667307795617e-09

Cosine similarity 4-letter w/ 0s for null guesses: 3.6572040527981194e-09
Cosine similarity 4-letter excluding null guesses: 3.951224688918701e-09

Cosine similarity 5-letter w/ 0s for null guesses: 6.680730762915937e-09
Cosine similarity 5-letter excluding null guesses: 7.833053933521561e-09

Cosine similarity 6-letter w/ 0s for null guesses: 7.012311150045956e-09
Cosine similarity 6-letter excluding null guesses: 9.023810762808143e-09

Cosine similarity 7-letter w/ 0s for null guesses: 4.967053731282552e-09
Cosine similarity 7-letter excluding null guesses: 7.450580596923828e-09

Cosine similarity 8-letter w/ 0s for null guesses: 5.676632835751488e-09
Cosine similarity 8-letter excluding null guesses: 8.66976651278409e-09

Cosine similarity 9-letter w/ 0s for null guesses: 0.0
Cosine similarity 9-letter excluding null guesses: 0.0

Cosine

In [ ]:
def is_correct_length(actual_answers, generated_answers):
  acc_array = np.array(actual_answers)
  gen_array = np.array(generated_answers)
  matches = np.sum(len(acc_array) == len(gen_array))
  return matches / len(acc_array)

In [ ]:
is_correct_length(actual_answers, generated_answers)

0.0004

In [ ]:
def is_correct_length(actual_answers, generated_answers):
    acc_array = np.array(actual_answers)
    gen_array = np.array(generated_answers)
    matches = np.sum([len(acc_array[i]) == len(gen_array[i]) for i in range(len(acc_array))])
    return matches / len(acc_array)

is_correct_length(words, guesses)

NameError: name 'words' is not defined

In [ ]:
# calculate accuracy by word
count = 0
for i in range(len(actual_answers)):
    word, guess = actual_answers[i], generated_answers[i]
    if word == guess:
        count += 1
print(f"Correct word prediction accuracy: {count / len(actual_answers)}")